# &nbsp; Gans Project: Cloud Functions

In [ ]:
#%pip install functions-framework

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import sqlalchemy
from lat_lon_parser import parse
from pytz import timezone
from datetime import datetime
import functions_framework

In [ ]:
# Define the API key to be passed into the url in a .py file called safekey.
from safe import API_key
from safe import sql_password

## 1.&nbsp; Weather cloud function, with testing

main.py file in GCP:

In [ ]:
import functions_framework
import pandas as pd
import requests
import sqlalchemy
from pytz import timezone
from datetime import datetime

@functions_framework.http
def weather_function(request):
    schema = "gans_schema" # The name of your database in cloud
    host = "34.78.116.238"
    user = "root"
    password = sql_password
    port = 3306
    
    # Create the connection string for SQLAlchemy
    connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
    
    # Read the cities table into a DataFrame
    cities_df = pd.read_sql("cities_info", con=connection_string)
    existing_weather_df = pd.read_sql("cities_weather", con=connection_string)
    
    berlin_timezone = timezone('Europe/Berlin')
    API_key = {API_key}
    
    # Define the limit to be passed into the url.
    limit = 5
    
    # Create an empty list for each of the values.
    weather_dictionary = {"city_id": [],
    "forecast_time": [],
    "outlook": [],
    "description": [],
    "temperature": [],
    "feels_like": [],
    "temp_min": [],
    "temp_max": [],
    "wind_speed": [],
    "rain_prob": []
    }
    
    # Get the coordinates of the desired city
    for _, city in cities_df.iterrows():
        city_id = city["city_id"]
        city_names = city["City"]
        coordinates = requests.get(f"http://api.openweathermap.org/data/2.5/forecast?q={city_names}&limit={limit}&appid={API_key}")
        coordinates_json= coordinates.json()
        
        # Define the parameters to be passed into the url.
        lat = coordinates_json["city"]["coord"]["lat"]
        lon = coordinates_json["city"]["coord"]["lon"]
        
        # Define the parameters to be passed into the url.
        weather = (requests
        .get(f"https://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&appid={API_key}&units=metric")
        )
        weather_json = weather.json()
        
        #retrieval_time = datetime.now(berlin_timezone).strftime("%Y-%m-%d %H:%M:%S")
        
        # Iterate over the list inside the weather_dictionary to get the data for different days and time.
        i = 0
        
        for _ in weather_json["list"]:
            # extract the relevant information
            weather_dictionary["city_id"].append(city_id)
            weather_dictionary["forecast_time"].append(weather_json["list"][i]["dt_txt"])
            weather_dictionary["outlook"].append(weather_json["list"][i]["weather"][0]["main"])
            weather_dictionary["description"].append(weather_json["list"][i]["weather"][0]["description"])
            weather_dictionary["temperature"].append(weather_json["list"][i]["main"]["temp"])
            weather_dictionary["feels_like"].append(weather_json["list"][i]["main"]["feels_like"])
            weather_dictionary["temp_min"].append(weather_json["list"][i]["main"]["temp_min"])
            weather_dictionary["temp_max"].append(weather_json["list"][i]["main"]["temp_max"])
            weather_dictionary["wind_speed"].append(weather_json["list"][i]["wind"]["speed"])
            weather_dictionary["rain_prob"].append(weather_json["list"][i]["pop"])
            #weather_dictionary["data_retrieved_at"].append(retrieval_time)
            i += 1
            
    # Create a DataFrame from the weather data
    weather_dict = pd.DataFrame(weather_dictionary)
    weather_dict["forecast_time"] = pd.to_datetime(weather_dict["forecast_time"])
    #weather_dict["data_retrieved_at"] = pd.to_datetime(weather_dict["data_retrieved_at"])

    weather_dict.to_sql('cities_weather', if_exists="replace", con=connection_string, index=False)

    return "Data has been sent to the table"


requirements.txt file:

````
functions-framework==3.*
sqlalchemy
PyMySQL
pandas
requests
pytz
````

We test our function by request_data. So we are going to simulate the http triggering of the cloud function.
* This code simulate the return statement of the function.
* And shows wether the function is going to run on the cloud or not.

In [ ]:
from flask import Request
import json

request_data = {}
request = Request.from_values(data=json.dumps(request_data))

# Call the function
response = weather_function(request)
print(response)

## 2.&nbsp; Flight CLOUD function, with testing

main.py file:

In [1]:
import functions_framework
import pandas as pd
from datetime import datetime, timedelta
import requests
from pytz import timezone
import sqlalchemy

@functions_framework.http
def flight_function(request):
    schema = "gans_schema" # The name of your database in cloud
    host = "34.78.116.238"
    user = "root"
    password = sql_password
    port = 3306
    
    # Create the connection string for SQLAlchemy
    connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

    airports_df = pd.read_sql("cities_airport", con=connection_string)
    existing_flights_df = pd.read_sql("cities_flight", con=connection_string)
    
    api_key = {API_key}
    berlin_timezone = timezone('Europe/Berlin')
    today = datetime.now(berlin_timezone).date()
    tomorrow = today + timedelta(days=1)
    # Create an empty list for each of the values.
    flight_dict_items = []

    # Get the icao of the desired airports
    for icaos in airports_df["airport_icao"]:
        times = [["00:00", "11:59"], ["12:00", "23:59"]]
        for time in times:
            url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icaos}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"
            querystring = {"withLeg":"false","direction":"Arrival","withCancelled":"false","withCodeshared":"true","withCargo":"false","withPrivate":"false","withLocation":"false"}
            headers = {
                "x-rapidapi-key": {API_key},
                "x-rapidapi-host": "aerodatabox.p.rapidapi.com"
                }
            response = requests.get(url, headers=headers, params=querystring)
            flight_json = response.json()

            retrieval_time = datetime.now(berlin_timezone).strftime("%Y-%m-%d %H:%M:%S")
            icao_id = airports_df.loc[airports_df["airport_icao"] == icaos, "airport_icao"].values[0]
            
            for item in flight_json["arrivals"]:
                flight_dict_item = {
                    "icao": icao_id,
                    "icao_arrival": item["movement"]["airport"].get("icao", None),
                    "arrival_local_time": item["movement"]["scheduledTime"].get("local", None),
                    "arrival_utc_time": item["movement"]["scheduledTime"].get("utc", None),
                    "revised_local_time": item["movement"].get("revisedTime", {}).get("local", 0),
                    "revised_utc_time": item["movement"].get("revisedTime", {}).get("utc", 0),
                    "flight_number": item.get("number", None),
                    "flight_status": item.get("status", None),
                    "airline_name": item["airline"].get("name", None),
                    }
                
                flight_dict_items.append(flight_dict_item)
        
    flight_dict_df = pd.DataFrame(flight_dict_items)

    flight_dict_df["arrival_local_time"] = pd.to_datetime(flight_dict_df["arrival_local_time"])
    # Remove the timezone
    flight_dict_df["arrival_local_time"] = flight_dict_df["arrival_local_time"].dt.tz_localize(None)
    # Add missing seconds before conversion
    flight_dict_df["revised_local_time"] = (flight_dict_df["revised_local_time"]
                                           .str
                                           .replace(r"(?<=\d{2}:\d{2})(?=\+)", ":00", regex=True)
                                           )
    # Convert to datetime
    flight_dict_df["revised_local_time"] = pd.to_datetime(flight_dict_df["revised_local_time"])
    # Remove the timezone
    flight_dict_df["revised_local_time"] = flight_dict_df["revised_local_time"].dt.tz_localize(None)
    #we can remove some columns if we don't need them
    flight_dict_df = (flight_dict_df
                      .drop(
                          columns=["arrival_utc_time","revised_utc_time","airline_name"]
                          )
                          )
    
    flight_dict_df.to_sql('cities_flight', if_exists='append',con=connection_string, index=False)

    return "Data has been sent to the table"


requirements.txt file:

````
functions-framework==3.*
pandas
sqlalchemy
requests
PyMySQL
````

Testing the function:

In [ ]:
from flask import Request
import json

request_data = {}
request = Request.from_values(data=json.dumps(request_data))

# Call the function
response = flight_function(request)
print(response)